In [1]:
import pandas as pd
import re
import rapidfuzz as rf

# Citire Fisier si analiza coloane

In [2]:
df = pd.read_parquet("veridion_entity_resolution_challenge.snappy.parquet")

In [3]:
pd.set_option('display.max_rows', 50)  


In [4]:
df.head(10)

,company_name,company_legal_names,company_commercial_names,main_country_code,main_country,main_region,main_city_district,main_city,main_postcode,main_street,...,generated_description,generated_business_tags,status,domains,all_domains,revenue,revenue_type,employee_count,employee_count_type,inbound_links_count
0,Owens Liquors,None,Owens Liquors,US,United States,South Carolina,None,Pawleys Island,29585,Ocean Highway,...,Owens Liquors is a retail establishment locate...,Retail Trade | Liquor Stores | Wine & Liquor,Active,None,None,None,None,None,None,None
1,Club Tarneit,None,Club Tarneit,AU,Australia,Victoria,Tarneit,City Of Wyndham,3029,None,...,None,None,Active,None,None,None,None,9,extracted,None
2,AAA Auto Otrokovice Zlín,None,AAA Auto Otrokovice Zlín,CZ,Czechia,Zlín,Kvítkovice U Otrokovic,Otrokovice,765 02,Zlínská,...,AAA Auto Otrokovice (Zlín) is a car dealership...,In-store Shopping | Investment Management Serv...,Active,None,None,None,None,None,None,None
3,Gisinger GmbH,Gisinger GmbH,None,DE,Germany,Baden-Württemberg,None,Ühlingen-Birkendorf,79777,Berauer Straße,...,None,None,Active,None,None,None,None,None,None,None
4,Kasana Life,None,Kasana Life,US,United States,Connecticut,None,Litchfield,06759,None,...,None,None,Active,None,None,None,None,None,None,None
5,BAM BROW & MAKEUP STUDIO,None,BAM BROW & MAKEUP STUDIO,AU,Australia,Western Australia,None,Mandurah,6201,None,...,None,None,Active,None,None,None,None,None,None,None
6,Tescoma,None,Tescoma,HU,Hungary,Budapest,None,Budapest,1222,Nagytétényi Út,...,Tescoma is a kitchen supply shop located in Bu...,Home Furnishings Retailer | Kitchenchen Supply...,Active,None,None,None,None,None,None,None
7,Happyweddings | No.1 Matrimony Trivandrum Kerala,None,Happyweddings | No.1 Matrimony Trivandrum Kera...,IN,India,Kerala,None,Thiruvananthapuram,695025,Medical College - Chalakkuzhy Road,...,Happyweddings.com is a company based in Thiruv...,Event Planning Services | Wedding Planning Ser...,Active,None,None,None,None,None,None,None
8,Dental Planet Manukau,None,Dental Planet Manukau,NZ,New Zealand,Auckland,None,Auckland,2104,Bakerfield Place,...,Dental Planet Manukau is a dental clinic locat...,Healthcare Services | Ddental Care Services,Active,None,None,None,None,None,None,None
9,Drake Design Photography,None,Drake Design Photography,US,United States,Texas,None,Lubbock,79416,County Road 6430,...,Drake Design Photography is a portrait studio ...,Wheelchair Access | Portrait Studio | Photogra...,Active,None,None,None,None,None,None,None


In [5]:
df.shape

(33446, 75)

In [6]:
df[df["created_at"].notna() & df["last_updated_at"].notna()][["created_at", "last_updated_at","company_name", "website_domain","main_country_code","main_city","main_region", ]].sort_values(by=["website_domain", "created_at", "last_updated_at"],ascending=[True, False,False])

,created_at,last_updated_at,company_name,website_domain,main_country_code,main_city,main_region
13755,2021-08-21 20:02:11,2022-05-30 22:55:36,AOWEI TECHNOLOGY,023zsys.com,RS,Belgrade,City Of Belgrade
19258,2023-02-08 15:11:46,2023-09-03 13:59:20,Keith Urban,0b8wwgg.shop,US,Nashville,Tennessee
29571,2024-08-11 23:46:53.785,2024-08-26 17:53:38.693,1300 Meteor Used Cars,1300meteor.com.au,AU,Townsville City,Queensland
20419,2022-04-27 07:47:03,2024-06-23 12:00:03.899,1300 Meteor Rentals Mount Isa,1300meteor.com.au,AU,Mount Isa City,Queensland
5130,2022-04-27 01:30:13,2024-08-11 14:42:09.075,1300 Meteor Rentals Cairns,1300meteor.com.au,AU,Cairns Regional,Queensland
...,...,...,...,...,...,...,...
23326,2019-11-24 01:22:21,2024-04-25 09:49:41,None,None,None,None,None
25119,2019-11-24 01:13:16,2024-07-13 09:21:27,None,None,None,None,None
11216,2019-11-24 00:13:17,2024-08-09 23:25:14,None,None,None,None,None
3492,2019-11-24 00:13:06,2024-07-17 14:24:22,None,None,None,None,None


In [7]:
#sa vedem ce putem folosi ca si cheie

info_df = pd.DataFrame({
    "dtype": df.dtypes,
    "non_null_count": df.notna().sum(),
    "non_null_percent": df.notna().mean() * 100
}).sort_values(by="non_null_count", ascending=False)

print(info_df)

                              dtype  non_null_count  non_null_percent
status                       object           33446        100.000000
last_updated_at              object           33403         99.871435
created_at                   object           33403         99.871435
company_name                 object           32617         97.521378
website_domain               object           31893         95.356694
...                             ...             ...               ...
naics_2022_secondary_labels  object             244          0.729534
android_app_url              object             138          0.412605
ios_app_url                  object             123          0.367757
alexa_rank                   object               0          0.000000
tiktok_url                   object               0          0.000000

[75 rows x 3 columns]


In [8]:
#nu ma ajuta la filtrare
df.status.unique()

array(['Active'], dtype=object)

In [9]:
#voi folosi company_name si website_domain ca si chei principale si voi elimina randurile care le au cu null pentru a avea uncitate
df = df.dropna(subset=["company_name", "website_domain"])


In [10]:
#ordonez pentru a avea cea mai recenta varianta
df = df.sort_values(
    by=[ "last_updated_at","created_at"],
    ascending=[False, False]
)

In [11]:
# verificam procentul de not nulluri per coloana pt company_name,website_domain
total = len(df)


for col in ["company_name", "website_domain"]:
    not_null_count = df[col].notna().sum()
    percent = (not_null_count / total) * 100
    print(f"{col}: {not_null_count}/{total} ({percent:.2f}%)")

company_name: 31154/31154 (100.00%)
website_domain: 31154/31154 (100.00%)


In [12]:
#verific numarul de randuri cu null pe created_at si last_updated_at

# cate au ambele NULL
both_null = df[(df["created_at"].isna()) & (df["last_updated_at"].isna())].shape[0]

# cate au NULL doar pe created_at
created_null = df[(df["created_at"].isna()) & (df["last_updated_at"].notna())].shape[0]

# cate au NULL doar pe last_updated_at
last_null = df[(df["created_at"].notna()) & (df["last_updated_at"].isna())].shape[0]

# cate le au pe ambele completate
both_notnull = df[(df["created_at"].notna()) & (df["last_updated_at"].notna())].shape[0]

print("Ambele NULL:", both_null)
print("Doar created NULL:", created_null)
print("Doar last_updated NULL:", last_null)
print("Ambele completate:", both_notnull)

Ambele NULL: 25
Doar created NULL: 0
Doar last_updated NULL: 0
Ambele completate: 31129


In [13]:
#le verific pe cele cu created_at si last_updated_at null
df_null_dates = df[(df["created_at"].isna()) & (df["last_updated_at"].isna())][["company_name", "website_domain"]]
df_null_dates

,company_name,website_domain
3230,Carolina Protective Services,onesecure.me
3330,YORBA LINDA FEED STORE INCORPORATED,yorbalindafeedstore.com
3517,Vins Bruchez,vinsbruchez.ch
3957,Peters & Associates,yourleaderinluxury.com
5191,Bastiaansen Wonen,bastiaansenwonen.nl
8286,Hh Druck,hhdruck.de
8513,KERÉKPÁRSZIGET,kerekpar-sziget.hu
10176,ESPRESSO MILANO,espressomilano.com
13303,Speight Luther Cpa,allriteindustries.com
16588,Kerokero store,kerokerostore.com.br


In [14]:
#concatenez randurile cu null pe created_at si last_updated_at cu randurile care se potrivesc pe acelasi domeniu pentru a vedea daca pierd date eliminand acele randuri
df_filled_dates = df[df["created_at"].notna() & df["last_updated_at"].notna()][["company_name", "website_domain","main_country_code","main_city", "created_at", "last_updated_at"]]

df_joined = df_null_dates.merge(
    df_filled_dates,
    on="website_domain",
    how="left",
    suffixes=("_null", "_filled")
)

display(df_joined)

,company_name_null,website_domain,company_name_filled,main_country_code,main_city,created_at,last_updated_at
0,Carolina Protective Services,onesecure.me,CAROLINA PROTECTIVE,US,Tampa,2023-07-05 19:45:07.045,2024-06-28 21:35:18
1,YORBA LINDA FEED STORE INCORPORATED,yorbalindafeedstore.com,YORBA LINDA FEED STORE INC.,US,Yorba Linda,2024-11-07 08:43:23.635,2024-12-02 20:14:59.436
2,YORBA LINDA FEED STORE INCORPORATED,yorbalindafeedstore.com,Yorba Linda Feed Store,US,Yorba Linda,2022-04-27 13:05:18,2024-10-23 08:22:44.783
3,YORBA LINDA FEED STORE INCORPORATED,yorbalindafeedstore.com,YORBA LINDA FEED STORE INC.,US,Yorba Linda,2024-07-25 04:54:17.322,2024-08-07 09:01:09.834
4,YORBA LINDA FEED STORE INCORPORATED,yorbalindafeedstore.com,YORBA LINDA FEED STORE INC.,US,Yorba Linda,2024-03-28 13:52:46.337,2024-03-28 13:52:46.337
...,...,...,...,...,...,...,...
111,Museo Nazionale Del Risorgimento Italiano Di T...,museorisorgimentotorino.it,Museo Nazionale del Risorgimento Italiano,IT,Turin,2023-02-06 13:31:48.698,2025-01-10 02:11:55.585
112,Museo Nazionale Del Risorgimento Italiano Di T...,museorisorgimentotorino.it,Museo Nazionale Del Risorgimento Italiano Di T...,IT,None,2024-08-20 13:57:17.83,2024-08-20 13:57:17.83
113,Museo Nazionale Del Risorgimento Italiano Di T...,museorisorgimentotorino.it,Museo Nazionale del Risorgimento,IT,Turin,2019-11-27 07:13:27,2024-07-28 17:19:41
114,Museo Nazionale Del Risorgimento Italiano Di T...,museorisorgimentotorino.it,Risorgimento Srl.,IT,Turin,2023-05-25 18:30:43.988,2024-04-25 07:50:03.21


In [15]:
#sterg liniile cu created_at si last_updated_at dupa ce m-am asigurat ca mai exista acea informatie
df = df.dropna(subset=["created_at", "last_updated_at"])


In [16]:
df.shape

(31129, 75)

In [17]:
# verificam procentul de not nulluri per coloana pt created_at,last_updated_at
total = len(df)


for col in ["created_at", "last_updated_at"]:
    not_null_count = df[col].notna().sum()
    percent = (not_null_count / total) * 100
    print(f"{col}: {not_null_count}/{total} ({percent:.2f}%)")

created_at: 31129/31129 (100.00%)
last_updated_at: 31129/31129 (100.00%)


In [18]:
df[["company_name", "website_domain","main_country_code","main_city", "created_at", "last_updated_at"]]

,company_name,website_domain,main_country_code,main_city,created_at,last_updated_at
5581,FinanceVue,finance-vue.fr,US,Los Angeles,2024-11-24 09:17:04,2025-01-22 03:25:12
9758,Orange City United Methodist Church,ocumc.org,US,Orange City,2021-07-08 21:40:47,2025-01-20 01:54:42.834
28236,Serious Designs,serious-designs.co.uk,GB,Stockton-on-Tees,2023-09-01 14:40:56.634,2025-01-19 09:34:54.577
29427,ACE physiotherapy bethnal green,acephysiotherapybethnalgreen.com,GB,London,2023-09-01 14:46:58.409,2025-01-18 22:42:28.139
16308,Smart Quality,smartquality.co.uk,GB,Bournemouth,2023-08-31 20:48:31.458,2025-01-18 17:17:29.75
...,...,...,...,...,...,...
16633,Genetic Builders Inc.,genbuilt.ca,CA,Winnipeg,2021-04-10 08:01:52,2021-04-10 08:01:52
4976,Daytona Homes Winnipeg,daytonahomes.ca,CA,Winnipeg,2021-04-10 07:52:04,2021-04-10 07:52:04
26944,Fotograaf norbert maes,norbertmaes.be,BE,Ruiselede,2021-04-02 13:49:41,2021-04-02 13:49:41
19340,The State Room,thestateroom.com,US,Salt Lake City,2021-04-02 13:40:10,2021-04-02 13:40:10


In [19]:
#sterg coloanele inutile pt ca doar imi ingreuneaza analiza au sub 2%
df.drop(columns=['naics_2022_secondary_codes','naics_2022_secondary_labels','android_app_url',
                 'ios_app_url','alexa_rank','tiktok_url','other_emails'],inplace=True)

# Normalizare date

1. company_name

In [20]:
#se poate vedea ca sunt mai putine unice daca fac lower.
df['company_name'].str.lower().nunique(), df['company_name'].nunique()

(16938, 18263)

In [21]:
df['company_name'].sample(40)  

15260                                           MERO IELTS
24331                                           Jaaya Axel
31591                      Saltro Hoofdlocatie Bloedafname
29772                       Atasca Health and Chiropractic
9904                              Colonial Court Motor Inn
22538                                     Waupun Utilities
5945                                       Seitz Stainless
22384                                        Brunswick BBQ
32611                        Wasco Nursery & Garden Center
31587                        Flow Engineering Technologies
26917                                      ST OSYTH parish
21415                                           Webromedia
24274                                       Radical Events
7448                                   CBA & PARTNERS BVBA
13114                Mudhumalai Forest Safari Bus Terminal
849                                        Ron OutSourcing
29217                          Menjačnica Trange-Frange 

In [22]:
stopwords = ['inc', 'llc', 'ltd', 'corp', 's.a.', 'srl', 'sa']

def normalize_company_name(name):
    if pd.isna(name):
        return None
    # lowercase + trim
    name = name.lower().strip()
    # eliminăm stopwords
    for word in stopwords:
        # cu spațiu înainte și după ca să nu taie părți din alte cuvinte
        name = re.sub(r'\b{}\b'.format(word), '', name)
    # eliminăm puncte, virgule, slash, multiple spații, minusuri
    name = re.sub(r'[-/&.,]', ' ', name)
    name = re.sub(r'\s+', ' ', name)  # un singur spațiu
    return name.strip()

In [23]:
#normalizez numele companiei
df['company_name_norm'] = df['company_name'].apply(normalize_company_name)

# verificăm un sample
df[['company_name', 'company_name_norm']].sample(20, random_state=42)

,company_name,company_name_norm
17589,Chola Gold,chola gold
9516,The Fireplace & Stove Shop,the fireplace stove shop
24335,Luces y Efectos Universales Vista Alegre Fuego...,luces y efectos universales vista alegre fuego...
22722,Tresmode,tresmode
18035,Hertfordshire Vehicle Services,hertfordshire vehicle services
25154,Funerária Narciso,funerária narciso
1219,Webwerk Krailing,webwerk krailing
29210,Allied Snow Plowing Removal,allied snow plowing removal
4576,Hospital Puerta de Hierro Tepic,hospital puerta de hierro tepic
18236,IAMHOCKEY,iamhockey


In [24]:
df['company_name_norm'].nunique()

16030

2. primary_phone

In [25]:
df['primary_phone'].sample(40) 

8137        +3613686571
17115    +4917629324939
11042     +573108023401
7698               None
5695       +12055472710
13889      +16613281410
21430      +18163641507
21449              None
32654      +48798827827
30593      +13053931092
27343      +13065635773
33020      +13303106119
33329     +917838888436
22003      +12285965678
11313     +441579321758
30314      +18883429227
7706     +4995617059360
20688     +886226956543
11037       +3234559360
563        +84904726173
6621      +390755052909
15501              None
27830     +442890351321
5558       +12297852025
30584              None
12080              None
25426     +441273757424
24516     +918028021717
15163       +4971919300
1165               None
24084    +4915221670750
5974       +19804461960
15895      +19053721409
33003      +41444314030
12672              None
8007       +19123506337
3789     +4369910049690
32831      +12033300130
32481              None
4848               None
Name: primary_phone, dtype: object

In [26]:
#scap de caractere non numerice
def normalize_phone(phone):
    if pd.isna(phone):
        return None
    return ''.join(filter(str.isdigit, str(phone)))

In [27]:
df['primary_phone_norm'] = df['primary_phone'].apply(normalize_phone)

In [28]:
df[['company_name_norm','primary_phone_norm','primary_phone']].sample(15)

,company_name_norm,primary_phone_norm,primary_phone
16030,pratra travel technologies,6531058632,+6531058632
11016,le boulevard restaurant,914132207500,+914132207500
7598,byb castellar soluciones,34647528959,+34647528959
32296,verniz arquitetura,None,None
3279,steam green carpet cleaning and restoration,13092827453,+13092827453
10035,hh kappers,31756287866,+31756287866
9815,digitales foerdern,497231203705,+497231203705
13043,tktx numbing cream tktx kr,17867961456,+17867961456
14230,fran ortiz,34625223431,+34625223431
10479,ellis cabinet and millwork,17818263600,+17818263600


3. website_domain


In [29]:
df['website_domain'].sample(30) 

4544           little-learners.net
17601           marianne-guely.com
14041          cmaclandscaping.com
31287         elbalcondepilatos.es
8775                    folio3.com
32150           ullevikonferans.se
23973           glidingclub.org.uk
25009     magicianforkidsparty.com
7319              giglogistics.com
11795             nelsonvets.co.uk
27201                northvolt.com
23378                doergcdzd.com
16321                   hellweg.de
11664        lajornadafilipina.com
24104       natalias-reisebuero.de
28960                     di-re.ru
24865               rpm-vision.com
2963         natur-erlebniswelt.de
4769            persimmonhomes.com
24919         diekunstbaustelle.de
28528                  gconinc.com
22884      hylananimalhospital.com
26036                 valantic.com
9653             divinesweetss.com
28401            bsc-heidelberg.de
156         joiningallmovement.com
5647                       naf.com
30917    degirmencioglubaklava.com
2792                

In [30]:
df['website_domain_norm'] = df['website_domain'].str.lower().str.strip()

In [31]:
#nu a facut nicio diferenta, deci datele sunt bune in formatul in care sunt acum
df['website_domain_norm'].nunique(), df['website_domain'].nunique()

(6558, 6558)

4. main_country_code

In [32]:
df['main_country_code'].sample(30) 

27190    None
14939      CH
25249      BG
9988       CL
3482       DE
6821       IN
32758      DE
18362    None
12332      GB
12436      VN
3296       GB
2497       BE
16965      DE
20562      IN
28825      US
25658      DE
22707      DE
22774      ES
3635       US
27521      NL
20593      IT
16804      ES
32657      US
8313       UA
31661      TR
597        MX
25567      CA
21681      NZ
23855    None
25504      PT
Name: main_country_code, dtype: object

In [33]:
df['main_country_code_norm']=df['main_country_code'].str.lower()

In [34]:
df['main_country_code_norm'].nunique(),df['main_country_code'].nunique()

(137, 137)

5. main_region

In [35]:
df['main_region'].sample(30) 

12276                                 Tbilisi
6941                                Occitania
15609                              California
10774                           North Brabant
25690                                 England
22829                                Scotland
14481                                 Florida
21537                                 England
21226                          North Carolina
32206                                  Oregon
9093                               Portalegre
11053                                 England
995                    North Rhine-Westphalia
32428                            Saskatchewan
15139                          Emilia-Romagna
22658                                 Florida
3784                               Tamil Nādu
3888                                 Oklahoma
8665                                 Colorado
12694                                   Dubai
1820                                  Alabama
31060                             

In [36]:
df['main_region_norm']=df['main_region'].str.lower().str.strip()

In [37]:
df['main_region_norm'].nunique(),df['main_region'].nunique()

(936, 936)

6. main_city

In [38]:
df['main_city'].sample(30) 

19878                  Gurugram
11967                   Havířov
11987                   Haarlem
6942                 Argenteuil
66                Mount Clemens
19955    Las Torres De Cotillas
32105                Callenberg
10024                   Jaroměř
25653                   Andover
20435                   Houston
10136                 Littleton
17517           East Longmeadow
8373                     Eugene
13675                   Marburg
23197                  Lithonia
2304                       None
6720                  Bengaluru
4678                      Leeds
12416    Sankt Veit An Der Glan
11196                      None
11346                      None
32193                 Aylesbury
6221                     Berlin
29785                  New York
21607                Manchester
20356                  Lakewood
30626             Kirchenlamitz
14742                    Verona
18372                    Monroe
12251                    Cuenca
Name: main_city, dtype: object

In [39]:
df['main_city_norm']=df['main_city'].str.lower().str.strip()

In [40]:
df['main_city_norm'].nunique(),df['main_city'].nunique()

(6490, 6490)

7. main_business_category

In [41]:
df['main_business_category'].sample(30) 

3079                                       None
5669     Home Builders & Renovation Contractors
33326                                    Hotels
14886                                      None
29591        Environmental Engineering Services
4664                                       None
33115        Automobile Dealers & Manufacturers
12119                             Party Rentals
11490                  Watches & Jewelry Stores
13311                                      None
235                         Roofing Contractors
20301                       Bakeries & Desserts
8061                                       None
4301                              Chiropractors
26568                 Computer Training Schools
21818                 Dentists & Dental Clinics
27602                                      None
5573                                       None
17847                              Sailing Club
14926                                      None
5820            Clinics - Surgeons & Phy

In [42]:
def normalize_business_category(name):
    if pd.isna(name):
        return None
    # lowercase + trim
    name = name.lower().strip()
    # eliminăm puncte, virgule, slash, multiple spații, minusuri
    name = re.sub(r'[-/&.,]', ' ', name)
    name = re.sub(r'\s+', ' ', name)  # un singur spațiu
    return name.strip()

In [43]:
#normalizez numele companiei
df['main_business_category_norm'] = df['main_business_category'].apply(normalize_business_category)

# verificăm un sample
df[['main_business_category_norm', 'main_business_category']].sample(20, random_state=42)

,main_business_category_norm,main_business_category
17589,watches jewelry stores,Watches & Jewelry Stores
9516,appliance stores,Appliance Stores
24335,missiles explosives fireworks,"Missiles, Explosives & Fireworks"
22722,shoes other footwear stores,Shoes & Other Footwear Stores
18035,None,None
25154,None,None
1219,None,None
29210,None,None
4576,clinics surgeons physicians,Clinics - Surgeons & Physicians
18236,None,None


In [44]:
df['main_business_category_norm'].nunique(),df['main_business_category'].nunique()

(539, 539)

8. main_postcode

In [45]:
df['main_postcode'].sample(30) 

4814          None
19530        46600
24315        57201
28039        37045
13711        33077
20291        35234
10563        21122
29886        04092
17896        76401
31080    65385-000
12845         None
1878        500085
1109          None
14508      CV1 2PS
5334       GL5 4BT
21010        89149
19271        09400
24407         None
20787         2440
27149        33064
21153       431002
17463         None
5185        010218
5635         78552
29116        49503
972        1824 EX
1165         49459
3919       CV9 3PL
30246         None
19676         None
Name: main_postcode, dtype: object

In [46]:
def normalize_postcode(code):
    if pd.isna(code) or str(code).strip().lower() == "none":
        return None
    
    code = str(code).strip().upper()
    
    # elimin spațiile
    code = re.sub(r"\s+", "", code)
    
    # elimin -
    if re.match(r"^\d{3}-\d{4}$", code):
        return code.replace("-", "")
    
    # inlocuieste tot ce nu e litera mare sau cifra cu nimic
    code = re.sub(r"[^A-Z0-9]", "", code)
    
    return code

In [47]:
df["main_postcode_norm"] = df["main_postcode"].apply(normalize_postcode)

In [48]:
#am corectat cateva
df['main_postcode_norm'].nunique(),df['main_postcode'].nunique()

(9888, 9946)

# Deduplicare Date

Varianta 1 : 2 coloane principale si restul secundare si cu scor(in functie de cate din aceste coloane sunt completate, ramane randul cu cele mai multe din ele completate)

In [49]:
secondary_cols = [
    
    'main_city_norm', 'main_region_norm',
    'main_country_code_norm', 'main_postcode_norm',
    'main_business_category_norm','primary_phone_norm'
]

def completeness_score(row):
    return sum([0 if pd.isna(row[col]) or row[col]=='' else 1 for col in secondary_cols])

df['completeness_score'] = df.apply(completeness_score, axis=1)

In [50]:
# Chei de grupare pentru deduplicare exactă
group_cols = ['company_name_norm', 'website_domain_norm' ]

# Alegem rândul cu cel mai mare scor de completitudine
df_dedup_1 = df.sort_values('completeness_score', ascending=False) \
             .drop_duplicates(subset=group_cols, keep='first') \
             .reset_index(drop=True)

In [51]:
# Prag de similaritate
threshold = 90
company_names = df_dedup_1['company_name_norm'].tolist()
fuzzy_groups = {}
visited = set()

for i, name in enumerate(company_names):
    if name in visited:
        continue
    matches = rf.process.extract(
        name, company_names, scorer=rf.fuzz.token_sort_ratio, score_cutoff=threshold)
    group_ids = [company_names.index(match[0]) for match in matches]
    for idx in group_ids:
        visited.add(company_names[idx])
        fuzzy_groups[company_names[idx]] = name  # grupăm sub numele principal

df_dedup_1['company_name_fuzzy_group'] = df_dedup_1['company_name_norm'].map(fuzzy_groups)

# Deduplicare finală cu fuzzy group + domain
group_cols_fuzzy = ['company_name_fuzzy_group', 'website_domain_norm']
df_final_1 = df_dedup_1.sort_values('completeness_score', ascending=False) \
                   .drop_duplicates(subset=group_cols_fuzzy, keep='first') \
                   .reset_index(drop=True)

print(f"Rânduri originale: {len(df)}")
print(f"Rânduri după deduplicare exactă: {len(df_dedup_1)}")
print(f"Rânduri după deduplicare fuzzy: {len(df_final_1)}")

Rânduri originale: 31129
Rânduri după deduplicare exactă: 17025
Rânduri după deduplicare fuzzy: 15893


Varianta 2: folosind o cheie mai ampla formata din company_name si website_domain + main_country_code,main_region,main_city 

In [52]:
df = df.dropna(subset=["main_city_norm", "main_region_norm","main_country_code_norm" ])


In [53]:
# verificam procentul de not nulluri per coloana pt main_city_norm,main_region_norm,main_country_code_norm
total = len(df)


for col in ["main_city_norm", "main_region_norm","main_country_code_norm"]:
    not_null_count = df[col].notna().sum()
    percent = (not_null_count / total) * 100
    print(f"{col}: {not_null_count}/{total} ({percent:.2f}%)")

main_city_norm: 27978/27978 (100.00%)
main_region_norm: 27978/27978 (100.00%)
main_country_code_norm: 27978/27978 (100.00%)


In [54]:
df.shape

(27978, 77)

In [55]:
# Chei de grupare pentru deduplicare exactă
group_cols = ['company_name_norm', 'website_domain_norm','main_city_norm', 'main_region_norm',
    'main_country_code_norm']

df_dedup_2 =df.drop_duplicates(subset=group_cols, keep='first') \
             .reset_index(drop=True)

In [ ]:
# Prag de similaritate
threshold = 90
company_names = df_dedup_2['company_name_norm'].tolist()
fuzzy_groups = {}
visited = set()

for i, name in enumerate(company_names):
    if name in visited:
        continue
    matches = rf.process.extract(
        name, company_names, scorer=rf.fuzz.token_sort_ratio, score_cutoff=threshold
    )
    group_ids = [company_names.index(match[0]) for match in matches]
    for idx in group_ids:
        visited.add(company_names[idx])
        fuzzy_groups[company_names[idx]] = name  # grupăm sub numele principal

df_dedup_2['company_name_fuzzy_group'] = df_dedup_2['company_name_norm'].map(fuzzy_groups)

# Deduplicare finală cu fuzzy group + domain
group_cols_fuzzy = ['company_name_fuzzy_group','website_domain_norm','main_city_norm', 'main_region_norm',
    'main_country_code_norm']
df_final_2 = df_dedup_2.drop_duplicates(subset=group_cols_fuzzy, keep='first') \
                   .reset_index(drop=True)

print(f"Rânduri originale: {len(df)}")
print(f"Rânduri după deduplicare exactă: {len(df_dedup_2)}")
print(f"Rânduri după deduplicare fuzzy: {len(df_final_2)}")